In [ ]:
import os
import sys
import gzip
import json
import urllib
import tempfile
import collections

import numpy as np
import pandas as pd
import networkx as nx

import requests
from bs4 import BeautifulSoup

from tqdm import tqdm_notebook as tqdm

from utils import load_config
from tad_helper_functions import *

In [ ]:
from tqdm import tqdm as tqdm_orig
tqdm_orig.pandas()

In [ ]:
config = load_config()

## Load data

In [ ]:
df_disgenet = pd.read_table(
    config['input_files']['raw_disgenet'],
    usecols=['snpId','diseaseId','diseaseName','source'])
df_disgenet['diseaseIdType'] = 'UMLS_CUI'

In [ ]:
df_disgenet.head()

## Decide whether to use hg19 or hg38

In [ ]:
# Restart kernel and rerun all cells after changing this cell
USE_HG38 = True

In [ ]:
if USE_HG38:
    genome_version = 'hg38'
    tad_data_fname = 'results/tads_hESC_hg38.tsv'
    
    def snp_position_convert(row):
        # don't do anything, as positions are already in hg38
        return row.position
else:
    genome_version = 'hg19'
    tad_data_fname = config['input_files']['tad_coordinates_hg19']
    
    # get hg19 SNP-positions
    df_snp_pos_map = pd.read_csv('results/snp_positions_hg19.csv')
    df_snp_pos_map['chrom'] = df_snp_pos_map['chrom'].apply(lambda x: x[3:])
    df_snp_pos_map['pos'] = list(zip(df_snp_pos_map['chrom'], df_snp_pos_map['end']))

    snp_pos_hg19 = df_snp_pos_map.set_index('SNPS').to_dict()['pos']
    def snp_position_convert(row):
        # convert BP-position from hg38 to hg19
        if row.snpId not in snp_pos_hg19:
            return np.nan
        
        chrom, pos_hg19 = snp_pos_hg19[row.snpId]
        assert row.chromosome == chrom
        return pos_hg19

## Integrate latest GWAS-catalog version

In [ ]:
df_gwascat = pd.read_table(config['input_files']['raw_gwascatalog'], low_memory=False)

df_gwascat = df_gwascat[['SNP_ID_CURRENT', 'MAPPED_TRAIT_URI', 'MAPPED_TRAIT']]
df_gwascat.dropna(inplace=True)
df_gwascat.rename(columns={
    'SNP_ID_CURRENT': 'snpId', 'MAPPED_TRAIT_URI': 'diseaseId', 'MAPPED_TRAIT': 'diseaseName'
}, inplace=True)

df_gwascat['snpId'] = df_gwascat['snpId'].apply(lambda x: f'rs{x}')
df_gwascat['source'] = 'GWASCUSTOM'
df_gwascat['diseaseId'] = df_gwascat['diseaseId'].str.split(' *, *').apply(
    lambda x: sorted(x)[0].split('/')[-1])
df_gwascat['diseaseIdType'] = df_gwascat['diseaseId'].apply(lambda x: x.split('_')[0])

df_gwascat.sample(5)

In [ ]:
df = pd.concat([df_disgenet, df_gwascat])
df.sample(5)

## Parse Ontology OWLs

In [ ]:
def parse_owl_file(soup, relevant_terms):
    """ Extract requested terms from OWL-file
    """
    node_owl_data = {}
    for entry in tqdm(soup.find_all('Class')):
        doid = entry['rdf:about'].split('/')[-1]

        # get label
        lbl = entry.find('rdfs:label').get_text()

        # get requested terms
        term_map = {term: [] for term in relevant_terms}
        for xref in entry.find_all('oboInOwl:hasDbXref'):
            txt = xref.get_text()
            for term in relevant_terms:
                if txt.startswith(f'{term}:'):
                    idx = txt.split(':')[-1]
                    term_map[term].append(idx)

        assert doid not in node_owl_data, doid
        node_owl_data[doid] = {
            'label': lbl,
            'terms': term_map
        }
        
    return node_owl_data

### DOID

In [ ]:
with open('data/doid.owl') as fd:
    soup_doid = BeautifulSoup(fd, 'xml')

In [ ]:
node_owl_data_doid = parse_owl_file(soup_doid, ['UMLS_CUI'])

## Disease ontology as tree

### Load data

In [ ]:
def load_ontology_network(type_):
    assert type_ in ('efo', 'doid'), f'Invalid type: {type_}'
    
    fname = f'cache/{type_}_graph.edgelist.gz'
    if not os.path.exists(fname):
        import onto2nx  # https://github.com/cthoyt/onto2nx
        nx.write_edgelist(onto2nx.parse_owl_rdf(f'data/{type_}.owl'), fname)
    else:
        print('Cached', fname)
        
    graph = nx.read_edgelist(fname, create_using=nx.DiGraph()).reverse()
    graph.name = type_
    return graph

In [ ]:
doid_graph = load_ontology_network('doid')
print(nx.info(doid_graph))

In [ ]:
efo_graph = load_ontology_network('efo')
print(nx.info(efo_graph))

### Map UMLS_CUI to DOID node

In [ ]:
doid_umls_map = {}
for node, data in tqdm(node_owl_data_doid.items()):
    assert set(data['terms'].keys()) == set(['UMLS_CUI'])
    doid_umls_map[node] = data['terms']['UMLS_CUI']

### Find cancer subtree

In [ ]:
given_diseases = set(df.diseaseId.unique())

# gather all possible disease-nodes
all_nodes_umls = [umls for doid in doid_graph.nodes() for umls in doid_umls_map[doid]]
all_nodes_efo = list(nx.descendants(efo_graph, 'EFO_0000408')) + ['EFO_0000408']  # disease subtree (vs traits, ...)

all_nodes = set(all_nodes_efo + all_nodes_umls) & given_diseases

# extract all cancer diseases
cancer_nodes_doid = (list(nx.descendants(doid_graph, 'DOID_162')) + ['DOID_162'])  # cancer subtree
cancer_nodes_umls = [umls for doid in cancer_nodes_doid for umls in doid_umls_map[doid]]

cancer_nodes_efo = list(nx.descendants(efo_graph, 'EFO_0000311')) + ['EFO_0000311']  # cancer subtree

cancer_nodes = set(cancer_nodes_efo + cancer_nodes_umls) & given_diseases
assert cancer_nodes <= all_nodes
print(f'#cancer nodes: {len(cancer_nodes)}/{len(all_nodes)}/{len(given_diseases)}')

In [ ]:
# do cancer-classification
data_cancer = []
for disease in tqdm(df['diseaseId'].unique()):
    if disease in all_nodes:
        data_cancer.append((disease, disease in cancer_nodes))
    
df_iscancer = pd.DataFrame(data_cancer, columns=['diseaseId','is_cancer'])
df_iscancer.sample(5)

## SNP annotations

### Retrieve VEP annotations

In [ ]:
def request_annotations(snps):
    _url = 'http://rest.ensembl.org/vep/human/id'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

    r = requests.post(_url, headers=headers, data=json.dumps({'ids': snps}))
    return r.json() if r.ok else None

In [ ]:
# warning: in case of update, cache-file must be deleted manually
fname = 'cache/snp_annotations.json'

if os.path.exists(f'{fname}.gz'):
    print('Cached', f'{fname}.gz')
    with gzip.open(f'{fname}.gz') as fd:
        snp_anno_data = json.load(fd)
else:
    # setup
    snp_anno_data = []

    batch_size = 200
    snp_list = df['snpId'].unique().tolist()

    # request annotations
    prev_i = 0
    for i in tqdm(range(batch_size, len(snp_list)+batch_size, batch_size)):
        i = min(i, len(snp_list))
        cur_snps = snp_list[prev_i:i]
        assert len(cur_snps) == (i-prev_i), (prev_i, i, len(cur_snps))

        res = request_annotations(cur_snps)
        assert res is not None
        snp_anno_data.extend(res)

        prev_i = i
        
    # cache results
    with open(fname, 'w') as fd:
        json.dump(snp_anno_data, fd)
    !gzip $fname

In [ ]:
snp_anno_extract = []
for e in snp_anno_data:
    snp_anno_extract.append((
        e['id'], e['most_severe_consequence'],
        e['seq_region_name'], e['start']
    ))
    
df_anno = pd.DataFrame(snp_anno_extract, columns=['snpId', 'variant_type', 'chromosome', 'position'])
df_anno.drop_duplicates('snpId', inplace=True)
df_anno.sample(5)

## Infer TAD relations

### Load SNP positions

In [ ]:
df_snppos = df_anno[['snpId', 'chromosome', 'position']].copy()
df_snppos.sample(5)

### Load TAD data

In [ ]:
df_tads = pd.read_table(tad_data_fname)

In [ ]:
df_tads.head()

### Do work

In [ ]:
def access_range_dict(row, dict_):
    range_dict_ = dict_.get(row['chromosome'], None)
    if range_dict_ is None:
        return 'undef'
    
    return range_dict_[row['position']]

In [ ]:
tad_anno_20in = parse_tad_annotations('20in', fname=tad_data_fname)
df_snppos['TAD_20in'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20in), axis=1)

tad_anno_40in = parse_tad_annotations('40in', fname=tad_data_fname)
df_snppos['TAD_40in'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40in), axis=1)

tad_anno_20out = parse_tad_annotations('20out', fname=tad_data_fname)
df_snppos['TAD_20out'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20out), axis=1)

tad_anno_40out = parse_tad_annotations('40out', fname=tad_data_fname)
df_snppos['TAD_40out'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40out), axis=1)

tad_anno_20inout = parse_tad_annotations('20inout', fname=tad_data_fname)
df_snppos['TAD_20inout'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20inout), axis=1)

tad_anno_40inout = parse_tad_annotations('40inout', fname=tad_data_fname)
df_snppos['TAD_40inout'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40inout), axis=1)

In [ ]:
df_snptads = df_snppos.drop(['chromosome', 'position'], axis=1)
df_snptads.sample(5)

## Merge into DisGeNET

In [ ]:
# individual databases
print('DisGeNET only:', df_disgenet.shape)
print('Most recent GWAS-catalog: ', df_gwascat.shape)

In [ ]:
# initial aggregation
df_final = df.copy()
df_final.shape

In [ ]:
# cancer-classification
df_final = df_final.merge(df_iscancer, on='diseaseId')
df_final.shape

In [ ]:
df_final = df_final.merge(df_snptads, on='snpId')
df_final.shape

In [ ]:
# SNP annotation
df_final = df_final.merge(df_anno, how='left')
df_final.shape

In [ ]:
# save to file
df_final.to_csv(f'results/disgenet_enhanced_{genome_version}.tsv', sep='\t', index=False)
df_final.sample(5)